## Classification

#### Classification is the process of classifying the objects from the input image using Classifier Network.

In [2]:
import torch.nn as nn 

In [ ]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, filter=2),
        )

        self.classification = nn.Sequential(
            nn.Flatten(),
            nn.Linear(16*5*5, 120),
            nn.Linear(120, 84),
            nn.Linear(84, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classification(x)
